In [18]:
import torch
from PIL import Image
from model import Network
from mobile_net import Mobile_Net_Unet
import cv2
import numpy as np
import time

In [19]:
checkpoint = torch.load("/home/akshay/DepthEstimation/checkpoint_28_kitti.pth")

In [20]:
model = Mobile_Net_Unet()
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print("Mobile Net Loaded")

Mobile Net Loaded


In [21]:
def to_tensor(image):
    if isinstance(image, np.ndarray):
        img = torch.from_numpy(image.transpose((2, 0, 1)))

    if image.mode == 'I':
        img = torch.from_numpy(np.array(image, np.int32, copy=False))
    elif image.mode == 'I;16':
        img = torch.from_numpy(np.array(image, np.int16, copy=False))
    else:
        img = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
    # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK

    if image.mode == 'YCbCr':
        nchannel = 3
    elif image.mode == 'I;16':
        nchannel = 1
    elif image.mode == 'RGBA':
        nchannel = 3
    else:
        nchannel = len(image.mode)

    img = img.view(image.size[1], image.size[0], nchannel)

    img = img.transpose(0, 1).transpose(0, 2).contiguous()

    if isinstance(img, torch.ByteTensor):
        return img.float().div(255)
    else:
        return img

In [24]:
image = Image.open("/home/akshay/Downloads/KITTI/data/train/X/2011_09_30_drive_0016_sync_0000000010.png")
target = Image.open("/home/akshay/Downloads/KITTI/data/train/target/2011_09_30_drive_0016_sync_0000000010.pngjpg")
left = int(image.size[0]/2-224/2)
upper = int(image.size[1]/2-192/2)
right = left + 224
lower = upper + 192
image = image.crop((left, upper, right,lower))
image.show()
target = target.crop((left, upper, right, lower))
target.show()

image = to_tensor(image).unsqueeze(axis=0)
pr = model(image).detach().numpy()
pr = np.transpose(pr[0], (1,2,0))
pr = 256 / pr
pr /= np.max(pr)
PIL_image = Image.fromarray(np.uint8(pr*255)).convert('RGB')
PIL_image.show()

In [8]:
nyu_image = Image.open("/home/akshay/Downloads/training/rgb/100.jpg")
image = nyu_image.resize((224,224))
image.show()
target.show()
img = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
img = img.view(image.size[1], image.size[0], 3)
img = img.transpose(0, 1).transpose(0, 2).contiguous()
if isinstance(img, torch.ByteTensor):
    img = img.float().div(255)
img1 = img.unsqueeze(axis=0)
pr = model(img1).detach().numpy()
pr = np.transpose(pr[0], (1,2,0))
pr = 1000 / pr
pr /= np.max(pr)
PIL_image = Image.fromarray(np.uint8(pr*255)).convert('RGB')
org = Image.open("/home/akshay/Downloads/training/depth/100.png")
org = org.resize((224,224))
org.show()
PIL_image.show()

In [5]:
checkpoint = torch.load("/home/akshay/DepthEstimation/checkpoint_24.pth")

In [6]:
model_vgg = Network()
model_vgg.load_state_dict(checkpoint['state_dict'])
model_vgg.eval()
print("VGG Loaded")

VGG Loaded


In [7]:
nyu_image = Image.open("/home/akshay/Downloads/training/rgb/100.jpg")
image = nyu_image.resize((224,224))
image.show()
img = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
img = img.view(image.size[1], image.size[0], 3)
img = img.transpose(0, 1).transpose(0, 2).contiguous()
if isinstance(img, torch.ByteTensor):
    img = img.float().div(255)
img1 = img.unsqueeze(axis=0)
pr = model(img1).detach().numpy()
pr = np.transpose(pr[0], (1,2,0))
pr = 1000 / pr
pr /= np.max(pr)
PIL_image = Image.fromarray(np.uint8(pr*255)).convert('RGB')
org = Image.open("/home/akshay/Downloads/training/depth/100.png")
org = org.resize((224,224))
org.show()
PIL_image.show()

In [8]:
image = Image.open("/home/akshay/DepthEstimation/test_dataset_generated/data/rgb/im5_Color_Color.png")

In [9]:
image = image.resize((224,224))
img = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))

In [10]:
img = img.view(image.size[1], image.size[0], 3)
img = img.transpose(0, 1).transpose(0, 2).contiguous()
if isinstance(img, torch.ByteTensor):
    img = img.float().div(255)
img1 = img.unsqueeze(axis=0)

In [11]:
start = time.time()
pr = model(img1).detach().numpy()
print("inference mbnet", time.time() - start)
pr = np.transpose(pr[0], (1,2,0))
pr = 35 / pr
pr /= np.max(pr)
PIL_image = Image.fromarray(np.uint8(pr*255)).convert('RGB')
PIL_image.show()

inference mbnet 0.021291017532348633


In [12]:
start = time.time()
pr = model_vgg(img1).detach().numpy()
print("inference mbnet", time.time() - start)
pr = np.transpose(pr[0], (1,2,0))
pr = 35 / pr
pr /= np.max(pr)
PIL_image = Image.fromarray(np.uint8(pr*255)).convert('RGB')
PIL_image.show()

inference mbnet 0.3208310604095459
